### 必要なパッケージをインポート

#### .envファイル
```yaml
OPENAI_API_KEY=sk-************************
```

In [7]:
import openai,os,requests,json,re,bs4
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

### Prompts

LangChainのRefineに倣い、InitialプロンプトとRefineプロンプトを作成した。\
不要な「,」を含んで、json.loads()でパースできない場合があったため、以下を追加した。\
（これでもたまにパースできないJsonを返してくる。。\
LangChainのOutput Parserを使いたいが、リストや入れ子構造があるJsonに対する方法がわからない。\
Pydantic Parserだと、英語で書かないとプロンプトが文字化けする。）

```
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。
```

また、Refineプロンプトでは、必要な場合のみ修正するようにした。
ドキュメントの最初に重要な情報が多く、最後に近づくにつれ関連コンテンツなど、メインコンテンツと関係ないものが多くなるという仮定を入れている。

```
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
```

In [9]:
## Initial Prompt
initial_str = """\
## Output Format
```json
{output_format}
```

## Input
{text}

## Instruction
[Input]はあるWebページのテキストの一部です。\
[Output Format]に従って出力してください。\
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。\
[Input]から読み取れないデータは出力に含めないでください。
"""

## Refine Prompt
refine_str = """\
## Output Format
```json
{output_format}
```

## Data
{data}

## Text
{text}

## Instruction
あなたのタスクは、最終的なデータを生成することです。\
現時点でのデータは[Data]です。
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
[Output Format]に従って出力してください。\
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。
"""
initial_prompt_template = ChatPromptTemplate.from_template(initial_str)
print('initial:',initial_prompt_template.messages[0].prompt.input_variables)
refine_prompt_template = ChatPromptTemplate.from_template(refine_str)
print('refine:',refine_prompt_template.messages[0].prompt.input_variables)

initial: ['output_format', 'text']
refine: ['data', 'output_format', 'text']


### ヘルパー関数

メイン関数で使う関数などを定義した。

`chunk_size`などは、現在はえいやで決めている。\
指示プロンプトと出力Jsonフォーマットのトークン数に基づき、トークン数制約ぎりぎりになるように調整することでチャンク数を抑えられるため、\
今後クラス化して使うときに調整機能をつけようと思う。

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3072,
    chunk_overlap  = 512,
    length_function = len,
)
chat = ChatOpenAI(temperature=0.)

def load_doc(url):
    response = requests.get(url)
    with open('temp.html','w') as f:
        f.write(response.text)
    loader = UnstructuredHTMLLoader("temp.html")
    data = loader.load()
    return data

def get_raw_html(url):
    response = requests.get(url)
    return response.text

def get_comressed_html(url):
    html = get_raw_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')

def extract_json(text,debug=False):
    patterns = [
        {"pattern": r'```json\n(.*?)```', "group": 1},
        {"pattern": r'\{.*\}', "group": 0},
    ]
    for p in patterns:
        match = re.search(p["pattern"], text, re.DOTALL)
        if match:
            json_string = match.group(p["group"])
            if debug:
                print('json_string:',json_string,sep='\n')
            data_json = json.loads(json_string)
            return data_json
    print('Error:Could not json extracted:',text)
    return {}

### メイン関数

まずはWebページのテキストをLangChainのドキュメントとし、チャンクに分割する。\
１つ目のチャンクにはInitialプロンプト、２つ目以降の各チャンクにはRefineプロンプトを適用する。\
最後にソースURLをデータに加えて終了。

In [11]:
def extact_json_from_url(url,data_format,debug=False):
    # doc = load_doc(url)
    # docs = text_splitter.split_documents(doc)
    text = get_raw_html(url)
    docs = text_splitter.create_documents([text])
    data = {}
    for i,doc in enumerate(docs):
        text = doc.page_content
        if debug:
            print(i+1,'-'*30)
            print(text.replace('\n','').replace(' ','')[:100],'...')
            print('...',text.replace('\n','').replace(' ','')[-100:])
        if i == 0:
            messages = initial_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        else:
            messages = refine_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text,
                data=json.dumps(data,indent=2,ensure_ascii=False)
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        if debug:
            print(json.dumps(data,ensure_ascii=False))
    data['url'] = url
    return data

### Example: レシピ

In [12]:
recipe_format = {
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
            "g": "int: 分量表記から推定した重量(gram)"
        }
    ],
    "steps": {
        "str: 手順番号":"str: 手順の説明"
    }
}

In [13]:
# url = "https://park.ajinomoto.co.jp/recipe/card/705645/"
url = "https://www.lettuceclub.net/recipe/dish/14360/"

In [14]:
data = extact_json_from_url(url,recipe_format)
print(json.dumps(data,indent=2,ensure_ascii=False))

### Example: イベント

In [ ]:
event_format = {
    'title': 'str: イベント名',
    'organizer': 'str: 主催者',
    'date': ['str: 日付。形式はYYYY/MM/DD'],
    'location': 'str: 開催場所',
    'description': 'str: イベント概要。400字以内',
    'schedule': ['str: スケジュール。形式はYYYY/MM/DD HH:MM~HH:MM（補足）'],
}

In [ ]:
# url = "https://food-innovation.co/sksjapan/sksj2023/"
url = "https://www.nagoya-info.jp/event/detail/404/"

In [ ]:
data = extact_json_from_url(url,event_format,debug=False)
print(json.dumps(data,indent=2,ensure_ascii=False))

{
  "title": "オアシス２１iセンター体験イベント＜有松・鳴海絞り染め＞",
  "organizer": "オアシス２１iセンター",
  "date": [
    "2023/06/12"
  ],
  "location": "オアシス２１iセンター前（オアシス２１地下1階）",
  "description": "名古屋には古くから染めものが根付いています。オアシス２１iセンターでは染めの楽しさを感じてもらえる「有松・鳴海絞り染め」の体験イベントを毎月開催しています。小さなお子様も大歓迎ですので、お気軽にご参加ください。英語対応も可能です。",
  "schedule": [
    "2023/06/12 10:00~18:00"
  ],
  "url": "https://www.nagoya-info.jp/event/detail/404/"
}
